In [2]:
import os
import zipfile
import shutil
import json
import time
import gc
from moviepy.editor import VideoFileClip
from tqdm import tqdm
import platform 

In [6]:
def process_videos_and_labels_in_zip(zip_file_path, output_folder, target_duration_minutes=10, labels_filename="labels.json"):
    """
    Unpacks videos and a JSON labels file from a ZIP, trims the videos,
    and saves them to a new folder. Handles videos in nested folders.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output folder: {output_folder}")

    temp_extract_folder = os.path.join(os.path.dirname(output_folder), "temp_extracted_videos")
    
    if os.path.exists(temp_extract_folder):
        print(f"Temporary folder exists: {temp_extract_folder}. Cleaning up...")
        try:
            shutil.rmtree(temp_extract_folder, ignore_errors=False, onerror=handle_remove_readonly)
        except Exception as e:
            print(f"Error cleaning up old temp folder: {e}. Please close any programs that might be using it or restart the script.")
            return
    os.makedirs(temp_extract_folder)
    print(f"Created temporary extraction folder: {temp_extract_folder}")

    print(f"Unpacking {zip_file_path}...")
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(temp_extract_folder)
        print("Unpacking complete.")
    except Exception as e:
        print(f"Critical error during ZIP extraction: {e}")
        return

    print("Starting video trimming and label processing...")

    processed_video_count = 0
    skipped_video_count = 0
    error_video_count = 0
    
    video_extensions = ('.mp4', '.avi', '.mkv', '.mov', '.webm', '.flv')
    files_to_process = []
    labels_file_path_in_temp = None

    for root, _, files in os.walk(temp_extract_folder):
        for filename in files:
            if filename.lower().endswith(video_extensions):
                files_to_process.append(os.path.join(root, filename))
            elif filename == labels_filename:
                labels_file_path_in_temp = os.path.join(root, filename)

    if labels_file_path_in_temp:
        try:
            shutil.copy2(labels_file_path_in_temp, os.path.join(output_folder, labels_filename))
            print(f"Labels file '{labels_filename}' copied to {output_folder}")
        except Exception as e:
            print(f"Error copying labels file '{labels_filename}': {e}")
    else:
        print(f"Warning: Labels file '{labels_filename}' not found in the ZIP archive.")

    for original_filepath in tqdm(files_to_process[5:], desc="Processing videos"):
        filename = os.path.basename(original_filepath)
        
        if filename.startswith("-"):
            safe_filename = "processed_" + filename
        else:
            safe_filename = filename
        
        output_filepath = os.path.join(output_folder, safe_filename)
        
        clip = None
        
        try:
            clip = VideoFileClip(original_filepath)
            target_duration_seconds = target_duration_minutes * 60

            write_params = {
                'codec': 'libx264',
                'audio_codec': 'aac',
                'threads': 4, 
                'preset': 'ultrafast',
                'logger': None
            }

            if clip.duration > target_duration_seconds:
                with clip.subclip(0, target_duration_seconds) as cut_clip:
                    cut_clip.write_videofile(output_filepath, **write_params)
                print(f"  Обрезано и сохранено: {safe_filename}")
                processed_video_count += 1
            else:
                clip.write_videofile(output_filepath, **write_params)
                print(f"  Видео {safe_filename} короче цели, скопировано.")
                skipped_video_count += 1
            
        except Exception as e:
            print(f"\n!!! Ошибка при обработке видео {filename}: {e}\n")
            error_video_count += 1
            if os.path.exists(output_filepath):
                try:
                    os.remove(output_filepath)
                    print(f"  Удален неполный выходной файл: {output_filepath}")
                except Exception as remove_e:
                    print(f"  Ошибка при удалении неполного файла {output_filepath}: {remove_e}")
        finally:
            if clip:
                try:
                    clip.close() 
                    if hasattr(clip, 'tempfiles'):
                        for temp_file in clip.tempfiles:
                            if os.path.exists(temp_file):
                                try:
                                    os.remove(temp_file)
                                except Exception:
                                    pass
                except Exception as close_e:
                    print(f"  Ошибка при закрытии clip ресурсов для {filename}: {close_e}")
            gc.collect()

    print("Attempting to remove temporary directory...")
    time.sleep(2)  
    try:
        shutil.rmtree(temp_extract_folder, ignore_errors=False, onerror=handle_remove_readonly)
        print(f"Temporary folder {temp_extract_folder} and its contents have been removed.")
    except Exception as e:
        print(f"ERROR: Failed to remove temporary folder {temp_extract_folder}: {e}")
        print("Please consider deleting it manually.")

    print("\n--- Processing Report ---")
    print(f"Total videos trimmed: {processed_video_count}")
    print(f"Total videos copied (shorter than target): {skipped_video_count}")
    print(f"Total errors during video processing: {error_video_count}")

def handle_remove_readonly(func, path, exc_info):
    
    if func == os.remove and platform.system() == "Windows":
        import stat
        os.chmod(path, stat.S_IWRITE)
        func(path)
    else:
        raise 

if __name__ == "__main__":
    zip_file = r"C:\Users\Admin\Downloads\data_train_short.zip"
    output_directory = r"D:\trimmed_videos_with_labels"
    labels_file_name_global = "labels.json"

    process_videos_and_labels_in_zip(
        zip_file, 
        output_directory, 
        target_duration_minutes=10, 
        labels_filename=labels_file_name_global
    )
    
    print(f"\nAll processed videos and the '{labels_file_name_global}' file are saved in: {output_directory}")


Temporary folder exists: D:\temp_extracted_videos. Cleaning up...
Created temporary extraction folder: D:\temp_extracted_videos
Unpacking C:\Users\Admin\Downloads\data_train_short.zip...
Unpacking complete.
Starting video trimming and label processing...
Labels file 'labels.json' copied to D:\trimmed_videos_with_labels


Processing videos:   0%|                                                                        | 0/75 [00:00<?, ?it/s]


!!! Ошибка при обработке видео -220020068_456241755.mp4: [Errno 32] Broken pipe

MoviePy error: FFMPEG encountered the following error while writing file D:\trimmed_videos_with_labels\processed_-220020068_456241755.mp4:

 



Processing videos:   1%|▊                                                          | 1/75 [34:37<42:41:40, 2077.04s/it]

  Удален неполный выходной файл: D:\trimmed_videos_with_labels\processed_-220020068_456241755.mp4


...

Processing videos:  99%|████████████████████████████████████████████████████████████▏| 74/75 [2:50:19<01:23, 83.96s/it]

  Обрезано и сохранено: processed_-220020068_456257139.mp4


Processing videos: 100%|████████████████████████████████████████████████████████████| 75/75 [2:51:43<00:00, 137.38s/it]

  Обрезано и сохранено: processed_-220020068_456257141.mp4
Attempting to remove temporary directory...


Temporary folder D:\temp_extracted_videos and its contents have been removed.

--- Processing Report ---
Total videos trimmed: 68
Total videos copied (shorter than target): 0
Total errors during video processing: 7

All processed videos and the 'labels.json' file are saved in: D:\trimmed_videos_with_labels
